In [ ]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns

from random import *

In [ ]:
def prob(y):
    if random() <= y:
        return 1
    else:
        return 0

In [ ]:
def trialgenerator(y):  #probabilistic environment: reward probability switches every 10-32 trials
    rewards = []
    true_values = []
    for trials in np.arange(50):
        for x in np.arange(randint(10,32)):
            rewards.append(prob(y))
            true_values.append(y)
        for x in np.arange(randint(10,32)):
            rewards.append(prob(1-y))
            true_values.append(1-y)
    del true_values[1000:]
    del rewards[1000:]
    return rewards, true_values

In [ ]:
def lever_update(alpha, value, reward):
    return value + alpha * (reward - value)

In [ ]:
def loopfunction(alpha, rewards, true_values):

    values = [0.5]
    for reward in rewards:
        values.append(lever_update(alpha, values[-1], reward))

    values.pop(0)

    return [abs(value - true_value) for value, true_value in zip(values, true_values)]

In [ ]:
correct_choices = {} 
for x in np.arange(0,1.05,0.05): #range of alphas
    x = round(x, 3)
    correct_choices[x] = {}
    for y in np.arange(0.55,1,0.05): #range of probabilities
        y = round(y, 3)
        correct_choices[x][y] = {}
        correctlist = []
        for n in range(1000):
            rewards, true_values = trialgenerator(y)
            correctlist.append(loopfunction(x, rewards, true_values))
        correct_choices[x][y] = correctlist

In [ ]:
heatmap = []
for y in np.arange(0.55,1,0.05):
    y = round(y, 3)
    heatmap.append([np.mean(correct_choices[x][y]) for x in correct_choices])

In [ ]:
df = pd.DataFrame(heatmap, index = ["55 / 45", "60 / 40", "65 / 35", "70 / 30", "75 / 25", "80 / 20", "85 / 15", "90 / 10", "95 / 5"], columns = [0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95,1] )
df = df.multiply(100) 

In [ ]:
ax = sns.heatmap(df, cmap="rocket_r")  
plt.xlabel("α values", fontsize = 12)
plt.ylabel("Reward probabilities (%)", fontsize = 12) 
ax.invert_yaxis()
cbar = ax.collections[0].colorbar
cbar.set_label('% error', labelpad=10, fontsize = 11)
plt.savefig('1d.png', bbox_inches='tight', dpi = 300) 